In [3]:
import sys
sys.path.append('/Applications/anaconda/lib/python2.7/site-packages')

In [4]:
import numpy as np
import pandas as pd
import nltk

In [5]:
import re
from nltk.corpus import stopwords # delete nonsense words like 'a' 'the'
from nltk.tokenize import RegexpTokenizer # split the sentence by break
from nltk.stem.snowball import SnowballStemmer # stemmer words
from nltk.stem.wordnet import WordNetLemmatizer # lemmatize
from nltk.corpus import wordnet
tokenizer = RegexpTokenizer(r'\.')
stemmer = SnowballStemmer('english')
st = WordNetLemmatizer()

def ngrams(input_list, n):
  return zip(*[input_list[i:] for i in range(n)])

def cleanTypo(word):
    if len(word) <= 2:
        return word.split(' ')
    else:
        isTypo = False
        index = 0
        char_list = list(word)
        three_chars = ngrams(char_list,3)
        for i in range(len(three_chars)):
            c1 = 'a' <= three_chars[i][0] <= 'z'
            c2 = 'A' <= three_chars[i][1] <= 'Z'
            c3 = 'a' <= three_chars[i][2] <= 'z'
            isTypo = isTypo or c1&c2&c3
            if isTypo:
                index = i+1
                cor_word = word[:index] + " " + word[index:]
                break

            else:
                cor_word = word

        return cor_word.split(" ")
    
def syno(word):
    if not word.isdigit():
        syn = wordnet.synsets(word)
        if syn:
            return str(syn[0].lemma_names()[0])
        else:
            return word  
    else:
        return word

def removeDot(s):
    if s == '.':
        return ' '
    elif s[0] == '.':
        s = s[1:]
    elif s[len(s)-1] == '.':
        s = s[:-1]
    return s    

In [6]:
def generateStr(sentence):
    text1 = []
    for i, item in enumerate(re.sub(r'[^\w\+\-\*\/\.\$]', ' ', sentence).split()):
        text1 = text1 + cleanTypo(item)
    text2 = [str(stemmer.stem(st.lemmatize(syno(word)))) for word in text1 if word.lower() not in stopwords.words('english')]
    text3 = ' '.join(removeDot(word) for word in text2)
    return text3

In [7]:
attr = pd.read_csv('attributes.csv')
attr.head(3)

,product_uid,name,value
0,100001,Bullet01,Versatile connector for various 90° connection...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001,Bullet03,Help ensure joints are consistently straight a...


In [8]:
brand = attr[attr['name']=='MFG Brand Name'].reset_index()
brand = brand.dropna()
brand.head()

,index,product_uid,name,value
0,9,100001,MFG Brand Name,Simpson Strong-Tie
1,37,100002,MFG Brand Name,BEHR Premium Textured DeckOver
2,69,100003,MFG Brand Name,STERLING
3,93,100004,MFG Brand Name,Grape Solar
4,122,100005,MFG Brand Name,Delta


In [9]:
color = attr[attr['name']=='Color/Finish'].reset_index()
color = color.dropna()
color.head()

,index,product_uid,name,value
0,31,100002,Color/Finish,Tugboat
1,59,100003,Color/Finish,White
2,115,100005,Color/Finish,Chrome
3,155,100006,Color/Finish,Stainless Steel
4,279,100010,Color/Finish,Silver/Gray


In [10]:
material = attr[attr['name']=='Material'].reset_index()
material = material.dropna()
material.head()

,index,product_uid,name,value
0,8,100001,Material,Galvanized Steel
1,67,100003,Material,Composite
2,202,100007,Material,Plastic
3,255,100009,Material,Medium Density Fiberboard (MDF)
4,282,100010,Material,Steel


In [11]:
brand['brand']=brand['value'].map(lambda x: generateStr(x))

In [13]:
brand1 = brand.iloc[:,[1,4]]
brand1.head(3)

,product_uid,brand
0,100001,simpson strong-ti
1,100002,behr premium textur deck
2,100003,sterl


In [14]:
len(brand1.product_uid)

86220

In [16]:
color['color']=color['value'].map(lambda x: generateStr(x))

In [17]:
color1 = color.iloc[:,[1,4]]
color1.head(3)

,product_uid,color
0,100002,tugboat
1,100003,white
2,100005,chrome


In [18]:
def check(a):
    print len(a)
    print len(a.unique())

In [21]:
material['material']=material['value'].map(lambda x: generateStr(x))

In [36]:
material1 = material.drop_duplicates('product_uid', keep='first')

In [37]:
material1 = material1.iloc[:,[1,4]]
material1.head(3)

,product_uid,material
0,100001,startl steel
1,100003,complex
2,100007,plastic


## Merge data

In [39]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')

In [40]:
train_attr = pd.merge(train,brand1,how='left',on='product_uid')

In [41]:
train_attr1 = pd.merge(train_attr,color1,how='left',on='product_uid')

In [42]:
train_attr2 = pd.merge(train_attr1,material1,how='left',on='product_uid')

In [45]:
train_attr2.head(3)

,id,product_uid,product_title,search_term,product_description,relevance,productList,searchList,desList,brand,color,material
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,"Not only do angles make joints stronger, they ...",3.0,simpson strong-ti 12-gaug angl,angl bracket,angl brand joint strong besid suppli consist h...,simpson strong-ti,NaN,startl steel
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,"Not only do angles make joints stronger, they ...",2.5,simpson strong-ti 12-gaug angl,liter bracket,angl brand joint strong besid suppli consist h...,simpson strong-ti,NaN,startl steel
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,BEHR Premium Textured DECKOVER is an innovativ...,3.0,behr premium textur deck 1-gal sc-141 tugboat ...,deck,behr premium textur deckov advanc solid color ...,behr premium textur deck,tugboat,NaN


In [46]:
train_attr2.to_csv('train_attr.csv',index=False)

In [47]:
test_attr = pd.merge(test,brand1,how='left',on='product_uid')
test_attr1 = pd.merge(test_attr,color1,how='left',on='product_uid')
test_attr2 = pd.merge(test_attr1,material1,how='left',on='product_uid')

In [48]:
test_attr2.head(3)

,id,product_uid,product_title,search_term,product_description,productList,searchList,desList,brand,color,material
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"Not only do angles make joints stronger, they ...",simpson strong-ti 12-gaug angl,90 degre bracket,angl brand joint strong besid suppli consist h...,simpson strong-ti,NaN,startl steel
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"Not only do angles make joints stronger, they ...",simpson strong-ti 12-gaug angl,metallic_el liter bracket,angl brand joint strong besid suppli consist h...,simpson strong-ti,NaN,startl steel
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able,"Not only do angles make joints stronger, they ...",simpson strong-ti 12-gaug angl,simpson sku abl,angl brand joint strong besid suppli consist h...,simpson strong-ti,NaN,startl steel


In [49]:
test_attr2.to_csv('test_attr.csv',index=False)